## Collapse Board Data to Firm Level, Merge Preqin Returns Data

### For Table 5 of Elite Embeddedness

By Charlie Eaton and Albina Gibadullina

In [321]:
set line 150
use "/Users/Charlie/Dropbox/Bankers in the Ivory Tower Book/2 Our New Financial Oligarchy/Data/Boards 2000 to 2017/board_members_all_long20220501.dta", clear
save d_board_members_all_long20220501, replace




file d_board_members_all_long20220501.dta saved


In [322]:
use unitid firmid firmname_new eigenvector betweenness strength schooltype schoolname year using d_board_members_all_long20220501, clear
drop if schooltype=="liberal arts"
drop if unitid==232186 | unitid==126818 | unitid==209542 | unitid==221759 | unitid==102614 | unitid==129020 | unitid==181464

**correct delaware unitid (flagship which has enrollment data etc.)
replace unitid=130943 if unitid==130343
keep if year>2002 & year<2018



(1,886 observations deleted)

(1,689 observations deleted)

(574 real changes made)

(4,765 observations deleted)


In [323]:
merge m:1 unitid year using d_school_level_elite_embed, keepusing(alumnipehedge ///
     acceptrate fte_count endowment_assets total03_revenue schooltypenum ///
    ftfirstefwhitt ptfirstefwhitt ftfirstefasiat ptfirstefasiat ftfirsteftotlt ptfirsteftotlt ///
    pe hedge otherfinance nonfinance fgrnt_p year cpi_scalar_2017)

tab schooltypenum

qui egen repfirst=rowtotal(ftfirstefwhitt ptfirstefwhitt ftfirstefasiat ptfirstefasiat)
qui egen allfirst=rowtotal(ftfirsteftotlt  ptfirsteftotlt)
qui gen pctunderfirst= (allfirst-allfirst) / allfirst

gen privatetop30=schooltypenum==3
gen publictop30=schooltypenum==2
gen rejectrtindex=1-acceptrate
gen rejectrtmn=1-acceptrate
gen rejectmax=1-acceptrate



    Result                      Number of obs
    -----------------------------------------
    Not matched                             0
    Matched                            26,346  (_merge==3)
    -----------------------------------------


schooltypen |
         um |      Freq.     Percent        Cum.
------------+-----------------------------------
     public |      6,585       24.99       24.99
    private |     19,761       75.01      100.00
------------+-----------------------------------
      Total |     26,346      100.00










In [324]:
collapse (max) rejectmax (first) schoolname firmname_new eigenvector betweenness strength cpi_scalar_2017 (sum) fte_count endowment_assets total03_revenue ///
    privatetop30 publictop30 pe hedge otherfinance nonfinance alumnipehedge rejectrtindex (mean) pctunderfirst fgrnt_p acceptrate ///
    rejectrtmn, by(firmid year)

. collapse (max) rejectmax (first) schoolname firmname_new eigenvector betweenn
> ess strength cpi_scalar_2017 (sum) fte_count endowment_assets total03_revenue
>      privatetop30 publictop30 pe hedge otherfinance nonfinance alumnipehedge 
> rejectrtindex (mean) pctunderfirst fgrnt_p acceptrate     rejectrtmn, by(firm
> id year)


In [325]:
qui gen endow_assets_c_log=log(endowment_assets / cpi_scalar_2017)
qui gen logtotal03_revenue=log(total03_revenue / cpi_scalar_2017)
qui gen logfte_count=log(fte_count)

*qui egen pehedge=rowtotal(pe hedge)
*qui egen boardcount=rowtotal(pe hedge otherfinance nonfinance)
*qui gen pctalt= pehedge/boardcount

label var alumnipehedge "# financier alumni in Forbes 400"
label var logfte_count "log FTE enrollment"
label var pctunderfirst "% underrepresented students of color"
label var logtotal03_revenue "log total revenue"
label var endow_assets_c_log "log endowment value"
label var fgrnt_p "% Pell recipients"
label var acceptrate "Admissions rate"

In [326]:
save d_firm_level_trustee_ties, replace

file d_firm_level_trustee_ties.dta saved


In [50]:
%head firstname lastname year firmid firmname_new eigenvector schoolname if firmname=="blackrock" & year==2017

,firstname,lastname,year,firmid,firmname_new,eigenvector,schoolname
32982,rick,rieder,2017,110,blackrock,.128855489306535,emory university
32983,robert,kapito,2017,110,blackrock,.128855489306535,university of pennsylvania
33006,laurence,fink,2017,110,blackrock,.128855489306535,new york university
33018,barbara,novick,2017,110,blackrock,.128855489306535,cornell university


In [333]:
import excel using Preqin_PEPerf_20210717195427.xlsx ///
, cellra(A22) case(lower) first clear 

. import excel using Preqin_PEPerf_20210717195427.xlsx , cellra(A22) case(lower
> ) first clear
(28 vars, 10,996 obs)


In [334]:
destring netirr benchnetirr netirrdiffpts, ignore("n" "/" "m" ",") replace


netirr: characters n / m removed; replaced as double
(2082 missing values generated)
benchnetirr: characters n / m removed; replaced as double
(970 missing values generated)
netirrdiffpts: characters n / m removed; replaced as double
(2113 missing values generated)


In [335]:
rename vintage year
drop if year<2003 | year>2017
tostring firmid, replace



(4,256 observations deleted)

firmid was long now str6


In [336]:
merge m:m firmid year using d_firm_level_trustee_ties, gen(trustmerge) keep(1 3)

. merge m:m firmid year using d_firm_level_trustee_ties, gen(trustmerge) keep(1
>  3)
(variable year was int, now double to accommodate using data's values)

    Result                      Number of obs
    -----------------------------------------
    Not matched                         6,278
        from master                     6,278  (trustmerge==1)
        from using                          0  (trustmerge==2)

    Matched                               462  (trustmerge==3)
    -----------------------------------------


In [337]:
destring firmid, replace
gen pctfundtarget= fundvaluemnusd / fundtargetvaluemnusd
replace fundvaluemnusd=fundvaluemnusd*1000000
replace fundtargetvaluemnusd=fundtargetvaluemnusd*1000000
gen lnfundvalue=ln(fundvaluemnusd)
gen lnfundtarget=ln(fundtargetvaluemnusd)


firmid: all characters numeric; replaced as long

(1,964 missing values generated)

(6,492 real changes made)

(4,878 real changes made)

(248 missing values generated)

(1,862 missing values generated)


In [338]:
foreach var in publictop30 privatetop30 eigenvector rejectrtindex rejectrtmn rejectmax {
    replace `var' = 0 if `var'==.
}
gen private=privatetop30!=0
gen public=publictop30!=0


(6,278 real changes made)
(6,278 real changes made)
(6,278 real changes made)
(6,278 real changes made)
(6,278 real changes made)
(6,278 real changes made)




In [339]:
save d_prequin_trustee_ties, replace

file d_prequin_trustee_ties.dta saved


In [273]:
set line 255
est clear

qui eststo: reg benchnetirr eigenvector   ///
    i.year , cluster(firmid)

qui eststo: reg benchnetirr  private public  ///
    i.year, cluster(firmid)

qui eststo: reg benchnetirr eigenvector private public  ///
    i.year, cluster(firmid)

qui eststo: reg benchnetirr eigenvector  ///
    i.year if year>2009 & year<2018, cluster(firmid)

qui eststo: reg benchnetirr private public  ///
    i.year if year>2009 & year<2018, cluster(firmid)

qui eststo: reg benchnetirr eigenvector private public  ///
    i.year if year>2009 & year<2018, cluster(firmid)

esttab, type cells(b(star fmt(3)) se(fmt(3)) par) stardetach  ///
	 legend label varlabels(_cons Constant)  stats(r2 N N_clust, fmt(3 0 0) ///
	label(R-square N Firms)) keep(eigenvector private public ) starlevels(^ .1 * .05 ** .01 *** .001)











--------------------------------------------------------------------------------------------------------------------------
                              (1)              (2)              (3)              (4)              (5)              (6)    
                     Bench Net ~)     Bench Net ~)     Bench Net ~)     Bench Net ~)     Bench Net ~)     Bench Net ~)    
                         b/se/par         b/se/par         b/se/par         b/se/par         b/se/par         b/se/par    
--------------------------------------------------------------------------------------------------------------------------
(first) eigenvector         4.061 ***                         2.382            6.658 ***                         4.352 *  
                            1.014                             1.815            1.747                             2.012    
                                                                                                                          
privat

In [ ]:
replace fundvaluemnusd=fundvaluemnusd*1000000
gen lnfundvalue=ln(fundvaluemnusd)
gen board=Number_of_board_members!=0
replace eigenvector=0 if eigenvector==.

In [52]:
import excel using Preqin_PEPerf_20210717195427.xlsx ///
, cellra(A22) case(lower) first clear 

. import excel using Preqin_PEPerf_20210717195427.xlsx , cellra(A22) case(lower
> ) first clear
(28 vars, 10,996 obs)


In [53]:
destring netirr benchnetirr netirrdiffpts, ignore("n" "/" "m" ",") replace


netirr: characters n / m removed; replaced as double
(2082 missing values generated)
benchnetirr: characters n / m removed; replaced as double
(970 missing values generated)
netirrdiffpts: characters n / m removed; replaced as double
(2113 missing values generated)


In [54]:
rename vintage year

In [55]:
sum fundvaluemnusd if status=="Liquidated"


    Variable |        Obs        Mean    Std. dev.       Min        Max
-------------+---------------------------------------------------------
fundvaluem~d |      3,622    374.3848    762.5525        .14    17708.4


In [56]:
collapse (rawsum) fundvaluemnusd fundtargetvaluemnusd (mean) netirr netirrdiffpts [aweight=fundvaluemnusd], by(firmid year)

. collapse (rawsum) fundvaluemnusd fundtargetvaluemnusd (mean) netirr netirrdif
> fpts [aweight=fundvaluemnusd], by(firmid year)


In [13]:
gen pctfundtarget=fundtargetvaluemnusd / fundvaluemnusd
replace fundvaluemnusd=fundvaluemnusd*1000000
gen lnfundvalue=ln(fundvaluemnusd)
gen board=Number_of_board_members!=0
replace eigenvector=0 if eigenvector==.

In [14]:
xtset firmid year
gen lyear=l.year
gen sinceprv=year-l.year



Panel variable: firmid (unbalanced)
 Time variable: year, 1980 to 2021, but with gaps
         Delta: 1 unit

(7,433 missing values generated)

(7,433 missing values generated)


In [15]:
drop if year<2003 | year>2017

(3,397 observations deleted)


In [16]:
tostring firmid, replace

firmid was long now str6


In [17]:
merge m:m firmid year using firm_measures, gen(trust_merge) keep(1 3)

(variable year was int, now double to accommodate using data's values)

    Result                      Number of obs
    -----------------------------------------
    Not matched                         5,333
        from master                     5,333  (trust_merge==1)
        from using                          0  (trust_merge==2)

    Matched                               290  (trust_merge==3)
    -----------------------------------------


In [356]:
*duplicates drop fundid year, force

In [18]:
replace Number_of_board_members=0 if trust_merge==1

(5,333 real changes made)


In [19]:
replace fundvaluemnusd=fundvaluemnusd*1000000
gen lnfundvalue=ln(fundvaluemnusd)
gen board=Number_of_board_members!=0
replace eigenvector=0 if eigenvector==.

(5,623 real changes made)


In [21]:
destring firmid, replace

firmid: all characters numeric; replaced as long


(0 real changes made)


In [23]:
duplicates drop firmid year, force


Duplicates in terms of firmid year

(1 observation deleted)


In [29]:
set line 255
est clear
xtset firmid year
qui eststo: reg netirr  Number_of_board_members  lnfundvalue i.year , cluster(firmid)
qui eststo: reghdfe netirr  Number_of_board_members lnfundvalue i.year, absorb(firmid) cluster(firmid)

esttab, type cells(b(star fmt(3)) se(fmt(3))) stardetach  ///
	 legend label varlabels(_cons Constant)  stats(r2 N N_clust, fmt(3 0 0) ///
	label(R-square N Firms)) keep(Number_of_board_members  ) starlevels(^ .1 * .05 ** .01 *** .001)





Panel variable: firmid (unbalanced)
 Time variable: year, 2003 to 2017, but with gaps
         Delta: 1 unit




------------------------------------------------------
                              (1)              (2)    
                     (mean) net~s     (mean) net~s    
                             b/se             b/se    
------------------------------------------------------
Number_of_board_me~s        0.534            1.136    
                            0.595            1.222    
------------------------------------------------------
R-square                    0.014            0.346    
N                            5051             4217    
Firms                        2032             1198    
------------------------------------------------------
^ p<.1, * p<.05, ** p<.01, *** p<.001


In [30]:
set line 255
est clear
xtset firmid year
qui eststo: reg netirr  eigenvector lnfundvalue i.year , cluster(firmid)
qui eststo: reghdfe netirr  eigenvector lnfundvalue i.year, absorb(firmid) cluster(firmid)

esttab, type cells(b(star fmt(3)) se(fmt(3))) stardetach  ///
	 legend label varlabels(_cons Constant)  stats(r2 N N_clust, fmt(3 0 0) ///
	label(R-square N Firms)) keep(eigenvector lnfundvalue) starlevels(^ .1 * .05 ** .01 *** .001)





Panel variable: firmid (unbalanced)
 Time variable: year, 2003 to 2017, but with gaps
         Delta: 1 unit




------------------------------------------------------
                              (1)              (2)    
                     (mean) net~s     (mean) net~s    
                             b/se             b/se    
------------------------------------------------------
eigenvector                 5.353            2.160    
                            4.942            5.181    
lnfundvalue                -1.158 ***       -2.004 ***
                            0.201            0.485    
------------------------------------------------------
R-square                    0.014            0.346    
N                            5051             4217    
Firms                        2032             1198    
------------------------------------------------------
^ p<.1, * p<.05, ** p<.01, *** p<.001


(5,333 real changes made)


In [411]:
replace betweenness=0 if betweenness==.
replace strength=0 if strength==.


(5,333 real changes made)

(5,333 real changes made)


In [421]:
set line 255
est clear
xtset firmid year
qui eststo: reg lnfundvalue eigenvector lncap_prv_5yrs netirrdiffpts i.year, cluster(firmid)
qui eststo: nbreg lnfundvalue eigenvector lncap_prv_5yrs netirrdiffpts i.year , cluster(firmid)
qui eststo: poisson lnfundvalue eigenvector lncap_prv_5yrs netirrdiffpts i.year , cluster(firmid)
qui eststo: reghdfe lnfundvalue eigenvector lncap_prv_5yrs netirrdiffpts i.year, absorb(firmid) cluster(firmid)

esttab, type cells(b(star fmt(3)) se(fmt(3))) stardetach  ///
	 legend label varlabels(_cons Constant)  stats(r2 N N_clust, fmt(3 0 0) ///
	label(R-square N Firms)) keep( eigenvector lncap_prv_5yrs ) starlevels(^ .1 * .05 ** .01 *** .001)





Panel variable: firmid (unbalanced)
 Time variable: year, 2003 to 2017, but with gaps
         Delta: 1 unit






----------------------------------------------------------------------------------------
                              (1)              (2)              (3)              (4)    
                      lnfundvalue      lnfundvalue      lnfundvalue      lnfundvalue    
                             b/se             b/se             b/se             b/se    
----------------------------------------------------------------------------------------
main                                                                                    
eigenvector                 0.204            0.002            0.002           -0.148    
                            0.464            0.023            0.023            0.460    
lncap_prv_5yrs              0.673 ***        0.034 ***        0.034 ***        0.051    
                            0.014            0.001            0.001            0

In [127]:
collapse (sum) fundvaluemnusd (mean) Number_of_board_members netirr netirrdiffpts [aweight=fundvaluemnusd], by(firmid year)

In [128]:
gen lnfundvalue=ln(fundvaluemnusd)

In [134]:
gen board=Number_of_board_members!=0

In [156]:
xtset firmid year
reghdfe netirr Number_of_board_members lnfundvalue, absorb(firmid year) cluster(firmid)



Panel variable: firmid (unbalanced)
 Time variable: year, 2003 to 2017, but with gaps
         Delta: 1 unit

(dropped 832 singleton observations)
(MWFE estimator converged in 8 iterations)

HDFE Linear regression                            Number of obs   =      4,225
Absorbing 2 HDFE groups                           F(   2,   1199) =      14.17
Statistics robust to heteroskedasticity           Prob > F        =     0.0000
                                                  R-squared       =     0.3784
                                                  Adj R-squared   =     0.1271
                                                  Within R-sq.    =     0.0092
Number of clusters (firmid)  =      1,200         Root MSE        =    18.8298

                              (Std. err. adjusted for 1,200 clusters in firmid)
-------------------------------------------------------------------------------
              |               Robust
       netirr | Coefficient  std. err.      t    P>|t|  

In [86]:
duplicates tag fundid year, gen(funddupe)


Duplicates in terms of fundid year


In [126]:
duplicates report firmid year


Duplicates in terms of firmid year

--------------------------------------
   Copies | Observations       Surplus
----------+---------------------------
        1 |         5117             0
        2 |          964           482
        3 |          327           218
        4 |          112            84
        5 |           95            76
        6 |           60            50
        7 |           42            36
       11 |           11            10
       12 |           12            11
--------------------------------------


In [118]:
tab year if lnfundvalue!=. & Number_of_board_members!=. & firmid!=. & fundid!=.


    Vintage |      Freq.     Percent        Cum.
------------+-----------------------------------
       2003 |        201        3.10        3.10
       2004 |        257        3.96        7.05
       2005 |        397        6.12       13.17
       2006 |        503        7.75       20.92
       2007 |        498        7.67       28.59
       2008 |        482        7.42       36.01
       2009 |        233        3.59       39.60
       2010 |        329        5.07       44.67
       2011 |        411        6.33       51.00
       2012 |        411        6.33       57.33
       2013 |        494        7.61       64.94
       2014 |        512        7.89       72.83
       2015 |        563        8.67       81.50
       2016 |        592        9.12       90.62
       2017 |        609        9.38      100.00
------------+-----------------------------------
      Total |      6,492      100.00


In [96]:
%browse if funddupe==1

,fundid,firmid,fundname,firmname,year,status,fundvaluemn,fundvaluemnusd,fundvaluemneur,fundtargetvaluemn,fundtargetvaluemnusd,fundtargetvaluemneur,type,buyoutsize,regionfocus,gplocation,called,distrdpi,valuervpi,netmultiplex,netirr,benchmarkspecifics,benchnetirr,netirrdiffpts,quartile,datereported,industryfocus,locationfocus,firmname_new,financetype,Number_of_board_members,strength,eigenvector,betweenness,trust_merge,lnfundvalue,_est_est1,_est_est2,_est_est3,funddupe
3426,31095,14794,Berkshire Multifamily Value Fund III,Berkshire Residential Investments,2013,Closed,471 USD,471000000,342.37,450 USD,450,350.87,Real Estate,.,US,US,103.4,135.7,24.9,1.61,12.2,2013 / US / Real Estate (Value Added),14.7,-2.5,3rd,9/30/2020,"Property, Hotels and Offices",US,berkshire group alt finance,Private equity,1,192,.106530657689443,3453.336272226822,Matched (3),19.970369,1,1,1,1


In [29]:
%browse if firmid==24391

,fundid,firmid,fundname,firmname,vintage,status,fundvaluemn,fundvaluemnusd,fundvaluemneur,fundtargetvaluemn,fundtargetvaluemnusd,fundtargetvaluemneur,type,buyoutsize,regionfocus,gplocation,called,distrdpi,valuervpi,netmultiplex,netirr,benchmarkspecifics,benchnetirr,netirrdiffpts,quartile,datereported,industryfocus,locationfocus
497,19079,24391,AGRE U.S. Real Estate Fund,Apollo Global Real Estate Management,2012,Closed,713 USD,713,542.38,650 USD,650,530.27,Real Estate,.,US,US,113.6,112.1,23.7,1.36,10.2,2012 / US / Real Estate (Opportunistic),12.7,-2.5,4th,12/31/2020,"Property, Hotels and Offices","Europe, US"
817,13312,24391,Apollo European Principal Finance Fund,Apollo Global Real Estate Management,2008,Closed,"1,400 EUR",1974.61,1400,"1,500 EUR",2115.66,1500,Real Estate,.,Europe,US,115.5,142.3,9.699999999999999,1.52,13,2008 / All Regions / Real Estate (Debt),8.699999999999999,4.3,1st,9/30/2020,Diversified,"Europe, West Europe"
818,27129,24391,Apollo European Principal Finance Fund II,Apollo Global Real Estate Management,2012,Closed,"2,500 EUR",3241.28,2500,"2,500 EUR",3064.48,2500,Real Estate,.,Europe,US,152,107.6,10,1.18,7.7,2012 / All Regions / Real Estate (Debt),8.800000000000001,-1.1,3rd,9/30/2020,Diversified,"Europe, West Europe"
819,62062,24391,Apollo European Principal Finance Fund III,Apollo Global Real Estate Management,2018,Closed,"3,844 EUR",4628.32,3844.1,"3,500 EUR",4074.6,3500,Real Estate,.,Europe,US,91.5,34.1,77.7,1.12,11.4,2018 / All Regions / Real Estate (Debt),8.1,3.3,1st,9/30/2020,Diversified,"Europe, Germany, Ireland, Italy, Spain, UK, We..."
839,51836,24391,Apollo U.S. Real Estate Fund II,Apollo Global Real Estate Management,2015,Closed,473 USD,473,421.06,750 USD,750,677.4,Real Estate,.,US,US,.,.,.,.,11,2015 / US / Real Estate (Value Added),12.1,-1.1,3rd,3/31/2021,"Property, Hotels and Offices",US
2864,8137,24391,CPI Capital Partners Europe,Apollo Global Real Estate Management,2006,Closed,"1,162 EUR",1533.49,1162,.,.,.,Real Estate,.,Europe,US,93.90000000000001,56.7,1.9,.59,-9.4,2006 / All Regions / Real Estate (Opportunistic),2.6,-12,4th,9/30/2020,"Property, Hotels and Offices","Germany, West Europe"
10164,13805,24391,Trophy Property Development,Apollo Global Real Estate Management,2008,Closed,"1,006 USD",1006,635.65,750 USD,750,476.25,Real Estate,.,Asia,US,98.3,73.5,.1,.74,-3.8,2008 / Asia / Real Estate (Opportunistic),1.2,-5,3rd,12/31/2020,"Property, Hotels and Offices",China


In [9]:
merge 1: firm_measures, clear

In [11]:
tab year


       year |      Freq.     Percent        Cum.
------------+-----------------------------------
       2003 |        110        4.10        4.10
       2004 |        128        4.77        8.86
       2005 |        135        5.03       13.89
       2006 |        150        5.58       19.47
       2007 |        156        5.81       25.28
       2008 |        182        6.78       32.06
       2009 |        181        6.74       38.79
       2010 |        185        6.89       45.68
       2011 |        200        7.45       53.13
       2012 |        206        7.67       60.80
       2013 |        213        7.93       68.73
       2014 |        216        8.04       76.77
       2015 |        209        7.78       84.55
       2016 |        206        7.67       92.22
       2017 |        209        7.78      100.00
------------+-----------------------------------
      Total |      2,686      100.00


In [37]:
use firm_measures, clear

In [38]:
duplicates tag firmid year, gen(dupe)


Duplicates in terms of firmid year


In [41]:
%browse if dupe==54

,firmname_new,financetype,year,Number_of_board_members,strength,eigenvector,betweenness,firmid,dupe
39,adacapital management,Venture capital,2003,1,100,.0515585586371593,0,.,54
106,ameriquest capital,Private equity,2003,1,50,.0074038770410941,0,.,54
164,aqr capital management,Hedge funds,2003,1,126,.0539632668323677,0,.,54
179,arcadian venture capital,Venture capital,2003,1,76,.0097573845214906,0,.,54
205,artemis capital partners,Hedge funds,2003,1,142,.5446597058076217,0,.,54
219,arthur rock and company,Venture capital,2003,1,126,.1611390455253501,0,.,54
382,bmd management company,Private equity,2003,1,70,.0407479797572614,0,.,54
434,brine management,Venture capital,2003,1,104,.0187823324882556,0,.,54
552,cedar street group,Venture capital,2003,1,104,.0187823324882556,0,.,54
613,chatham capital corp,Private equity,2003,1,14,0,0,.,54
